In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
#TICKERS = ['QCOM', 'NVDA', 'AMZN', 'MSFT', 'GOOG', 'TSLA', 'AMD', 'INTC', 'NFLX', 'BAC', 'WFC', 'GS', 'MA', 'SQ', 'PYPL']
TICKERS = ['NVDA']
frequency = 15

In [ ]:
results_df = pd.DataFrame(columns=['Ticker', 'Mid_Price_Std', 'Cumulative_Return'])

In [ ]:
for ticker in TICKERS:
    # Read the pickle file for the stock
    stock = pd.read_pickle(f'../data/pickles/{ticker}.pickle.gzip')

    stock = stock.dropna()


    match ticker:
        case 'NVDA':
            split_stock = True
            split_date = '2021-07-20'
            split_ratio = 4/1
        case 'AMZN':
            split_stock = True
            split_date = '2022-06-06' # +2
            split_ratio = 20/1
        case 'GOOG':
            split_stock = True
            split_date = '2022-07-17' # +2
            split_ratio = 20/1
        case 'TSLA':
            split_stock = True
            split_date = '2022-08-25'
            split_ratio = 3/1
        case _  :
            split_stock = False

    # Calculate mid price and spread
    stock['mid'] = (stock['P'] + stock['p']) / 2
    
    if split_stock:
        stock_split_date = pd.Timestamp(split_date)
        before_split = stock.index < stock_split_date
        stock.loc[before_split, 'mid'] /= split_ratio

    stock = stock.replace([np.inf, -np.inf], np.nan).dropna()

    # Compute standard deviation of mid price
    mid_price_std = stock['mid'].std()

    # Calculate cumulative return
    stock['percentage_return'] = stock['mid'].pct_change() * 100

    stock = stock.replace([np.inf, -np.inf], np.nan).dropna()
    
    cumulative_return = stock['percentage_return'].iloc[1:].sum()

    # Create a DataFrame with the current result
    current_result = pd.DataFrame([{'Ticker': ticker,
                                     'Mid_Price_Std': mid_price_std,
                                     'Cumulative_Return': cumulative_return}])

    # Concatenate the current result to the overall results DataFrame
    results_df = pd.concat([results_df, current_result], ignore_index=True)

In [ ]:
results_df

In [ ]:
stock['mid'].pct_change().iloc[1:].describe()